# Spatial Join for Land Use and Sensor Readings

In [63]:
import pandas as pd
import numpy as np
import geopandas as gp
import sys
import matplotlib.pyplot as plt
%pylab inline
print sys.version

Populating the interactive namespace from numpy and matplotlib
2.7.11 (default, Dec  5 2015, 14:44:47) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.1.76)]


In [64]:
# read in dataset containing all unique points
# not sure what to do with this - can someone please clarify?
uniloc = pd.read_csv('UniqueLocations.csv')

In [65]:
print uniloc.head()
print 'We have %d unique locations' % len(uniloc)

   Unnamed: 0          uniqueLatLon    GPS_LAT    GPS_LON  ID
0           0  42.223632|-71.169822  42.223632 -71.169822  24
1           1  42.223827|-71.166122  42.223827 -71.166122  18
2           2  42.223926|-71.167671  42.223926 -71.167671  30
3           3  42.224193|-71.172439  42.224193 -71.172439  20
4           4  42.225318|-71.175933  42.225318 -71.175933  44
We have 152033 unique locations


In [66]:
# read in dataset containing all points - do we need to roll these up to the unique locations?
df = pd.read_csv('nUrve_master.csv')
df.head()

,ID,GPS_DATETIMESTAMP,GPS_LAT,GPS_LON,GPS_Speed,GPS_Alt,GPS_Sats,GPS_Fix,GPS_Quality,AMB_Temp,...,RDQ_AcXMea,RDQ_AcY,RDQ_AcYMin,RDQ_AcYMax,RDQ_AcYMea,RDQ_AcZ,RDQ_AcZMin,RDQ_AcZMax,RDQ_AcZMea,SamplingCount
0,132,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.23,...,-2.4004,0.3530,0.1569,1.0199,0.5672,9.1006,7.9238,9.2183,8.6780,255
1,134,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.23,...,-2.3879,0.3138,0.1569,0.8630,0.5544,9.1790,8.2376,9.3359,8.7051,254
2,136,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.24,...,-2.4036,0.7453,0.1177,0.9414,0.5233,8.3945,8.1199,9.1790,8.6665,255
3,138,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.23,...,-2.3473,0.6669,0.2354,1.0199,0.5401,8.5514,8.1591,9.1006,8.6938,254
4,140,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.23,...,-2.4079,0.5492,0.0392,0.9414,0.5836,8.6299,8.0022,9.5321,8.6882,255


In [67]:
len(df)

1058495

In [70]:
# read in land use using a shapefile
landuse=gp.read_file('landuse/GISDATA_LANDUSE_POLY.shp')
landuse.head(1)

,AREA_ACRES,LU21_1971,LU21_1985,LU21_1999,LU37_1985,LU37_1999,LU_ID,RELATEID,TILE_NAME,geometry
0,11.7899,1,1,1,1,1,338,19900338,199,POLYGON ((-71.26024364338319 42.25648712959588...


In [71]:
# import necessary modules for the spatial join
import os
from shapely.geometry import Point
from geopandas import GeoDataFrame, read_file
from geopandas.tools import overlay
from geopandas.tools import sjoin

In [73]:
# combine lat/lon pairs from sensor data into actual points
df['geometry'] = df.apply(lambda x: Point(x.GPS_LON, x.GPS_LAT), axis =1)
df.head(1)

,ID,GPS_DATETIMESTAMP,GPS_LAT,GPS_LON,GPS_Speed,GPS_Alt,GPS_Sats,GPS_Fix,GPS_Quality,AMB_Temp,...,RDQ_AcY,RDQ_AcYMin,RDQ_AcYMax,RDQ_AcYMea,RDQ_AcZ,RDQ_AcZMin,RDQ_AcZMax,RDQ_AcZMea,SamplingCount,geometry
0,132,2015-10-26 2:48:45.0,42.34143,-71.080329,0.47,183.7,4,1,1,21.23,...,0.353,0.1569,1.0199,0.5672,9.1006,7.9238,9.2183,8.678,255,POINT (-71.08032900000001 42.34143)


In [74]:
# df = GeoDataFrame(df,crs = {'init': 'epsg:4326'})

# def crsMatch(x,y):
#     if x == y:
#         print 'the crs values match!'
#     else:
#         "the crs values don't match!"

# crsMatch(landuse.crs,df.crs)

the crs values match!


In [76]:
# spatial join
join = sjoin(df, landuse, how='left', op='within')

In [78]:
len(join)

1058495